# Big G Express: Predicting Derates
In this project, you will be working with fault code data and vehicle onboard diagnostic data to try and predict an upcoming full derate. These are indicated by an SPN 5246. 
 
Note that in its raw form the data does not have "labels", so you must define what labels you are going to use and create those labels in your dataset. Also, you will likely need to perform some significant feature engineering in order to build an accurate predictor.

When evaluating the performance of your model, assume that the cost associated with a missed full derate is approximately $4000.00$ in towing and repairs, and the cost of a false positive prediction is about $500 due to having the truck off the road and serviced unnecessarily.

A failed component is usually what triggers this code.

Common Failures

 * Failed DEF doser valve
 * Associated fault code: SPN 5394
 * You ran out of DEF fluid
 * Associated fault code: SPN 5392, SPN 1761
 * Inlet and Outlet NOx sensors failed, not making pressure
 * Associated fault code: SPN 4094
 * EGR system malfunction causing NOX efficiency problems
 * DEF pump failed, not making pressure
 * Associated fault code: SPN 4334, SPN 4339
 * DEF module has failed or DEF harness failure or no power to DEF module causes DEF gauge to be empty and showing datalink error and SCR malfunction.
 * The DEF / ECM could also need updating to eliminate ghost codes.

In this notebook, we will:
 * load and compare features between the preprocessed notebooks spns/fmi and the emission control spns/fmi
 * Generate onehotencoded columns for the top faultIDs
 * and other preprocessing that needs to be done, in preparation for fitting the ml model

In [1]:
import pandas as pd
import numpy as np
import re
import datetime as dt
import missingno as msno
from datetime import timedelta
import matplotlib.pyplot as plt

In [2]:
faults_processed = pd.read_csv('../data/faults_merged_pt1_done.csv', low_memory=False)
faults_processed.head(2)

,Unnamed: 0,RecordID,EventTimeStamp,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,...,EngineTimeLtd,FuelLtd,FuelRate,IgnStatus,IntakeManifoldTemperature,LampStatus,Speed,TurboBoostPressure,geometry,distance
0,0,1,2015-02-21 10:47:13,111,17,True,2,1439,38.86,-84.63,...,1632.2,12300.907429328,0,False,78.8,1023,0,0,POINT (2995931.112894146 658983.4394280221),322511.48
1,5,6,2015-02-21 11:40:22,111,17,True,1,1417,33.04,-96.18,...,9480,70349.809963756,4.583399,True,111.2,1023,13.6022,6.67,POINT (2203912.075923482 -273868.7810324882),1241142.92


#### For some reason, the df reads in with columns that are mixed types. Need to reset some column dtypes.

In [3]:
#For some reason, the df reads in with columns that are mixed types. Need to reset some column dtypes.
# fix data types
faults_processed=faults_processed.drop(columns=['Unnamed: 0'])
faults_processed['EventTimeStamp'] = pd.to_datetime(faults_processed['EventTimeStamp'])
faults_processed['LocationTimeStamp'] = pd.to_datetime(faults_processed['LocationTimeStamp'])

In [4]:
faults_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 507342 entries, 0 to 507341
Data columns (total 30 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   RecordID                   507342 non-null  int64         
 1   EventTimeStamp             507342 non-null  datetime64[ns]
 2   spn                        507342 non-null  int64         
 3   fmi                        507342 non-null  int64         
 4   active                     507342 non-null  bool          
 5   activeTransitionCount      507342 non-null  int64         
 6   EquipmentID                507342 non-null  object        
 7   Latitude                   507342 non-null  float64       
 8   Longitude                  507342 non-null  float64       
 9   LocationTimeStamp          507342 non-null  datetime64[ns]
 10  time_of_day                507342 non-null  object        
 11  BarometricPressure         507342 non-null  object  

In [5]:
sfc = pd.read_excel("../data/Service Fault Codes_1_0_0_167.xlsx")
sfc.head(2)

C:\Users\micha\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
0,Y,111,167,Not Mapped,254,0,12,629,12,P0606,Red,Stop / Shutdown,Engine Control Module Critical Internal Failur...,Error internal to the ECM related to memory ha...
1,Y,112,167,Not Mapped,20,128,7,635,7,Not Mapped,Red,Stop / Shutdown,Engine Timing Actuator Driver Circuit - Mechan...,Mechanical failure in the engine timing actuat...


In [6]:
sfc=sfc.rename(columns={'Lamp Device':'lamp_device', 
                        'SPN':'spn',
                        'Cummins Description':'cummins_descrip',
                       'Algorithm Description':'algo',
                        'J1939 FMI':'fmi'
                       })

In [7]:

keywords = ['NOx', 'Aftertreatment', 'Emissions', 'DEF', 'Oxidation', 'Catalyst', 'Decomposition', 'SCR', 'DPF']

def filter_function(description):
    return any(re.search(r'\b{}\b'.format(keyword), description, flags=re.IGNORECASE) for keyword in keywords)

sfc_emission_faults = sfc[sfc['cummins_descrip'].apply(filter_function)]

In [8]:
#969 rows.  
sfc_emission_faults.head(2)

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,spn,fmi,J2012 Pcode,Lamp Color,lamp_device,cummins_descrip,algo
1061,Y,1663,167,Not Mapped,326,128,13,3241,13,P141A,Amber,Warning,Aftertreatment Exhaust Gas Temperature 1 Swapp...,The aftertreatment diesel oxidation catalyst i...
1062,Y,1664,167,Not Mapped,380,128,11,4796,31,Not Mapped,Amber,Warning,Aftertreatment 1 Diesel Oxidation Catalyst Mis...,The aftertreatment diesel oxidation catalyst i...


In [9]:
sfc_emission_faults[["spn", "fmi"]]=sfc_emission_faults[["spn", "fmi"]].apply(pd.to_numeric)

C:\Users\micha\AppData\Local\Temp\ipykernel_6036\1603515747.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sfc_emission_faults[["spn", "fmi"]]=sfc_emission_faults[["spn", "fmi"]].apply(pd.to_numeric)


In [10]:
sfc_emission_faults['spn_combined']=sfc_emission_faults['spn'].astype(str)+'_'+ sfc_emission_faults['fmi'].astype(str)


C:\Users\micha\AppData\Local\Temp\ipykernel_6036\317711213.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sfc_emission_faults['spn_combined']=sfc_emission_faults['spn'].astype(str)+'_'+ sfc_emission_faults['fmi'].astype(str)


In [11]:
sfc_emission_faults.info()

<class 'pandas.core.frame.DataFrame'>
Index: 969 entries, 1061 to 5796
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Published in CES 14602  969 non-null    object
 1   Cummins Fault Code      969 non-null    int64 
 2   Revision                969 non-null    int64 
 3   PID                     969 non-null    object
 4   SID                     969 non-null    object
 5   MID                     969 non-null    object
 6   J1587 FMI               969 non-null    int64 
 7   spn                     969 non-null    int64 
 8   fmi                     969 non-null    int64 
 9   J2012 Pcode             969 non-null    object
 10  Lamp Color              836 non-null    object
 11  lamp_device             836 non-null    object
 12  cummins_descrip         969 non-null    object
 13  algo                    190 non-null    object
 14  spn_combined            969 non-null    object
dtypes: int6

In [12]:
sfc_emission_faults.head(2)

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,spn,fmi,J2012 Pcode,Lamp Color,lamp_device,cummins_descrip,algo,spn_combined
1061,Y,1663,167,Not Mapped,326,128,13,3241,13,P141A,Amber,Warning,Aftertreatment Exhaust Gas Temperature 1 Swapp...,The aftertreatment diesel oxidation catalyst i...,3241_13
1062,Y,1664,167,Not Mapped,380,128,11,4796,31,Not Mapped,Amber,Warning,Aftertreatment 1 Diesel Oxidation Catalyst Mis...,The aftertreatment diesel oxidation catalyst i...,4796_31


In [13]:

faults_processed['spn_combined'] = faults_processed['spn_combined']=faults_processed['spn'].astype(str)+'_'+ faults_processed['fmi'].astype(str)

In [14]:
faults_processed.head()

,RecordID,EventTimeStamp,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,LocationTimeStamp,...,FuelLtd,FuelRate,IgnStatus,IntakeManifoldTemperature,LampStatus,Speed,TurboBoostPressure,geometry,distance,spn_combined
0,1,2015-02-21 10:47:13,111,17,True,2,1439,38.86,-84.63,2015-02-21 11:34:25,...,12300.907429328,0,False,78.8,1023,0,0,POINT (2995931.112894146 658983.4394280221),322511.48,111_17
1,6,2015-02-21 11:40:22,111,17,True,1,1417,33.04,-96.18,2015-02-21 11:40:59,...,70349.809963756,4.583399,True,111.2,1023,13.6022,6.67,POINT (2203912.075923482 -273868.7810324882),1241142.92,111_17
2,7,2015-02-21 11:40:52,111,17,True,2,1597,36.90,-86.44,2015-02-21 11:41:29,...,40961.065436834,14.29175,True,78.8,1023,41.53478,20.59,POINT (2924633.055684726 399381.7435326362),302008.99,111_17
3,11,2015-02-21 11:42:19,111,17,True,1,1582,40.16,-80.15,2015-02-21 11:42:55,...,37724.957799834,13.72378,True,78.8,1023,64.82649,17.69,POINT (3294822.680030754 940816.0324587803),513749.92,111_17
4,15,2015-02-21 11:14:38,1067,2,True,127,309,36.18,-86.70,2015-02-21 11:44:52,...,9487.342989502,0,True,100.4,1279,26.31119,0.58,POINT (2930614.391331102 316197.3132442413),316187.13,1067_2


In [15]:
#Example: You need to pull out the spn_fmi series from each dataframe and compare the two 

series1 = pd.Series([1, 2, 3, 4, 5, 9])
series2 = pd.Series([3, 4, 5, 6, 7, 9])

# Find values that are common in both series
common_values = series1[series1.isin(series2)]

print(common_values)

2    3
3    4
4    5
5    9
dtype: int64


In [16]:
faults_series=faults_processed.spn_combined
sfc_series=sfc_emission_faults.spn_combined

common_faultcodes=faults_series[faults_series.isin(sfc_series)]

unique_faults=common_faultcodes

In [17]:
unique_faults

32        4364_18
82         3226_4
95        4364_18
121        5848_4
139       3936_15
           ...   
507315     1761_9
507316    1761_19
507317     3031_9
507333    3362_31
507340    3216_10
Name: spn_combined, Length: 20585, dtype: object

In [18]:
one_hot_encoded = pd.get_dummies(unique_faults)

In [19]:
one_hot_encoded

,1761_1,1761_10,1761_11,1761_17,1761_18,1761_19,1761_3,1761_4,1761_9,3031_18,...,5862_3,5862_4,6773_16,6780_3,6802_31,7321_4,7323_4,7854_2,7854_3,7854_4
32,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
82,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
95,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
121,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
139,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507315,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
507316,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
507317,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
507333,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [20]:
faults_processed.loc[30:35]

,RecordID,EventTimeStamp,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,LocationTimeStamp,...,FuelLtd,FuelRate,IgnStatus,IntakeManifoldTemperature,LampStatus,Speed,TurboBoostPressure,geometry,distance,spn_combined
30,104,2015-02-21 12:55:52,50353,0,True,2,1432,35.89,-84.74,2015-02-21 12:56:28,...,67281.187407724,0,True,46.4,2,52.94277,0.87,POINT (3106399.922361057 348228.4870717321),145026.01,50353_0
31,109,2015-02-21 13:01:31,111,17,True,1,1600,36.48,-84.22,2015-02-21 13:02:08,...,36166.078520982,0,True,53.6,1023,55.48651,2.32,POINT (3126063.34307579 426361.9602302434),98980.72,111_17
32,112,2015-02-21 13:02:17,4364,18,True,1,1442,35.20,-86.69,2015-02-21 12:57:08,...,68055.34360611,14.96539,True,96.8,17407,49.27279,20.3,POINT (2968390.08398245 213428.3950517494),336897.95,4364_18
33,113,2015-02-21 13:04:16,111,17,True,1,1377,36.03,-85.09,2015-02-21 13:04:53,...,73889.58337453,18.21472,True,59,1023,52.93306,26.39,POINT (3071717.927490912 351083.5323678879),173162.18,111_17
34,117,2015-02-21 12:43:18,929,9,True,126,1630,35.80,-86.40,2015-02-21 13:06:20,...,33322.794725306,0.9906483,True,64.4,1279,0,0,POINT (2970299.753093609 285813.7243969291),293501.49,929_9
35,122,2015-02-21 13:09:56,929,9,True,126,1605,36.94,-86.49,2015-02-21 13:10:31,...,32191.609998642,0.4887198,True,86,1279,3.52434,0.29,POINT (2919165.862244155 401619.5793119175),307216.81,929_9


In [21]:
merged_df = pd.merge(faults_processed, one_hot_encoded, how='left', left_index=True, right_index=True)
merged_df

,RecordID,EventTimeStamp,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,LocationTimeStamp,...,5862_3,5862_4,6773_16,6780_3,6802_31,7321_4,7323_4,7854_2,7854_3,7854_4
0,1,2015-02-21 10:47:13,111,17,True,2,1439,38.86,-84.63,2015-02-21 11:34:25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,2015-02-21 11:40:22,111,17,True,1,1417,33.04,-96.18,2015-02-21 11:40:59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,2015-02-21 11:40:52,111,17,True,2,1597,36.90,-86.44,2015-02-21 11:41:29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11,2015-02-21 11:42:19,111,17,True,1,1582,40.16,-80.15,2015-02-21 11:42:55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15,2015-02-21 11:14:38,1067,2,True,127,309,36.18,-86.70,2015-02-21 11:44:52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507337,1248448,2020-03-06 13:12:43,96,3,True,126,1936,30.38,-81.74,2020-03-06 13:29:33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
507338,1248452,2020-03-06 13:42:48,111,18,True,93,1886,39.02,-77.03,2020-03-06 13:43:24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
507339,1248455,2020-03-06 14:04:23,1569,31,True,5,1994,34.39,-79.46,2020-03-06 14:04:59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
507340,1248456,2020-03-06 14:13:38,3216,10,True,1,1850,34.43,-84.92,2020-03-06 14:14:14,...,False,False,False,False,False,False,False,False,False,False


In [22]:
merged_df.fillna(False, inplace=True)

In [23]:
merged_df.head(2)

,RecordID,EventTimeStamp,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,LocationTimeStamp,...,5862_3,5862_4,6773_16,6780_3,6802_31,7321_4,7323_4,7854_2,7854_3,7854_4
0,1,2015-02-21 10:47:13,111,17,True,2,1439,38.86,-84.63,2015-02-21 11:34:25,...,False,False,False,False,False,False,False,False,False,False
1,6,2015-02-21 11:40:22,111,17,True,1,1417,33.04,-96.18,2015-02-21 11:40:59,...,False,False,False,False,False,False,False,False,False,False


In [24]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 507342 entries, 0 to 507341
Columns: 242 entries, RecordID to 7854_4
dtypes: bool(214), datetime64[ns](2), float64(4), int64(6), object(16)
memory usage: 211.9+ MB


In [25]:
merged_df.columns

Index(['RecordID', 'EventTimeStamp', 'spn', 'fmi', 'active',
       'activeTransitionCount', 'EquipmentID', 'Latitude', 'Longitude',
       'LocationTimeStamp',
       ...
       '5862_3', '5862_4', '6773_16', '6780_3', '6802_31', '7321_4', '7323_4',
       '7854_2', '7854_3', '7854_4'],
      dtype='object', length=242)

In [28]:
merged_df.to_csv('../data/big_g_pipeline_ready.csv')